###  Gerador de Horários de Ponto para Jornada de 8 Horas Diárias

- **4 registros por dia:** Cada dia contém *quatro horários de marcação de ponto*: **Entrada**, **Saída para Almoço**, **Retorno do Almoço** e **Saída Final**. Assim, há dois intervalos principais (manhã e tarde) para cada dia trabalhado.

- **Jornada total de 8 horas:** O gerador calcula os horários de forma que a **soma do tempo trabalhado** (manhã + tarde, excluindo o intervalo de almoço) seja de **8 horas**. Ou seja, a duração do trabalho pela manhã somada à da tarde totaliza 8h. O intervalo de almoço não conta como tempo trabalhado.



In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

def gerar_horarios_dia():
    # Entrada entre 08:00 e 09:15
    entrada = datetime.strptime("08:00", "%H:%M") + timedelta(minutes=random.randint(0, 75))

    # Turno da manhã: entre 3h e 4h (180 a 240 minutos)
    duracao_manha = random.randint(180, 240)
    saida_almoco = entrada + timedelta(minutes=duracao_manha)

    # Intervalo de almoço: entre 60 e 90 minutos
    intervalo_almoco = random.randint(60, 90)
    retorno_almoco = saida_almoco + timedelta(minutes=intervalo_almoco)

    # Término do dia: completar 8h totais de trabalho
    duracao_tarde = 480 - duracao_manha  # 8h = 480 minutos
    saida_final = retorno_almoco + timedelta(minutes=duracao_tarde)

    # Total trabalhado
    total_trabalhado = timedelta(minutes=duracao_manha + duracao_tarde)

    return {
        "Entrada 1": entrada.strftime("%H:%M"),
        "Saída 1": saida_almoco.strftime("%H:%M"),
        "Entrada 2": retorno_almoco.strftime("%H:%M"),
        "Saída 2": saida_final.strftime("%H:%M"),
        "Banco de Horas": f"{total_trabalhado.seconds // 3600:02d}:{(total_trabalhado.seconds % 3600) // 60:02d}",
        "Observação": "" if random.random() > 0.1 else "Este registro foi inserido manualmente."
    }

# Gerar 10 dias úteis consecutivos
dados = [gerar_horarios_dia() for _ in range(30)]

# Criar DataFrame
df = pd.DataFrame(dados)

# Salvar em Excel
caminho_arquivo = "E:/data-engineering-projects/registro_ponto_simulado.xlsx"
df.to_excel(caminho_arquivo, index=False)

caminho_arquivo


'E:/data-engineering-projects/registro_ponto_simulado.xlsx'

In [4]:
import pandas as pd
import random
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

def gerar_horarios_dia(data_base):
    # Entrada entre 08:00 e 09:15
    entrada = datetime.combine(data_base, datetime.min.time()) + timedelta(hours=8) + timedelta(minutes=random.randint(0, 75))

    # Variação total do dia (±3 minutos)
    variacao_total = random.randint(-3, 3)

    # Turno da manhã: entre 3h30 e 4h15 (210 a 255 minutos)
    duracao_manha = random.randint(210, 255)
    saida_almoco = entrada + timedelta(minutes=duracao_manha)

    # Intervalo de almoço: entre 60 e 90 minutos
    intervalo_almoco = random.randint(60, 90)
    retorno_almoco = saida_almoco + timedelta(minutes=intervalo_almoco)

    # Término do dia: completar 8h totais de trabalho com variação
    duracao_total = 480 + variacao_total
    duracao_tarde = duracao_total - duracao_manha
    saida_final = retorno_almoco + timedelta(minutes=duracao_tarde)

    # Total trabalhado
    total_trabalhado = timedelta(minutes=duracao_manha + duracao_tarde)
    horas = total_trabalhado.seconds // 3600
    minutos = (total_trabalhado.seconds % 3600) // 60

    # Observações eventuais
    observacao = ""
    if random.random() < 0.15:
        observacao = random.choice([
            "Este registro foi inserido manualmente.",
            "Atraso justificado.",
            "Saída antecipada autorizada.",
            "Almoço prolongado com aprovação.",
        ])

    return {
        "Data": data_base.strftime("%Y-%m-%d"),
        "Entrada 1": entrada.strftime("%H:%M"),
        "Saída 1": saida_almoco.strftime("%H:%M"),
        "Entrada 2": retorno_almoco.strftime("%H:%M"),
        "Saída 2": saida_final.strftime("%H:%M"),
        "Banco de Horas": f"{horas:02d}:{minutos:02d}",
        "Observação": observacao
    }

# Gerar dias úteis a partir de hoje
numero_dias = 15
data_inicial = datetime.today()
datas_uteis = [data_inicial + BDay(i) for i in range(numero_dias)]

# Gerar dados
dados = [gerar_horarios_dia(data.date()) for data in datas_uteis]

# Criar DataFrame
df = pd.DataFrame(dados)

# Adicionar linha de totalizador
def somar_horas(col):
    total = timedelta()
    for h in col:
        t = datetime.strptime(h, "%H:%M")
        total += timedelta(hours=t.hour, minutes=t.minute)
    horas = total.seconds // 3600
    minutos = (total.seconds % 3600) // 60
    return f"{horas:02d}:{minutos:02d}"

totalizador = {
    "Data": "Total",
    "Entrada 1": "",
    "Saída 1": "",
    "Entrada 2": "",
    "Saída 2": "",
    "Banco de Horas": somar_horas(df["Banco de Horas"]),
    "Observação": "Total de horas trabalhadas"
}
df.loc[len(df.index)] = totalizador

# Exportar para Excel e CSV
excel_path = "E:/data-engineering-projects/registro_ponto_melhorado.xlsx"
csv_path = "E:/data-engineering-projects/registro_ponto_melhorado.csv"

df.to_excel(excel_path, index=False)
df.to_csv(csv_path, index=False)

excel_path, csv_path


('E:/data-engineering-projects/registro_ponto_melhorado.xlsx',
 'E:/data-engineering-projects/registro_ponto_melhorado.csv')